In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import shutil

Extract the zip file:

In [2]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [4]:
zip_name = 'fruits-small.zip'
extracted_file = 'fruits-small'

# Make new directory:
os.makedirs(extracted_file, exist_ok = True)

try:
    shutil.unpack_archive(filename= zip_name, extract_dir= extracted_file)
    print('Successfully extacted the file')
except Exception as e:
    print('Error while unpacking the zip file.')

Successfully extacted the file


Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [5]:
# Make data compatible using ImageDataGenerators:
# Each image in the Folders will be normalized using rescale.
train_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.0)

In [6]:
# Splitting Test and Train Data:

train_image_data = train_image_generator.flow_from_directory('fruits-small/fruits-small/data/Training',
                                                             batch_size= 3,
                                                             class_mode = 'categorical',
                                                             target_size = (224,224))


test_image_data = test_image_generator.flow_from_directory('fruits-small/fruits-small/data/Validation',
                                                           batch_size= 3,
                                                           class_mode = 'categorical',
                                                           target_size= (224,224))

Found 3425 images belonging to 7 classes.
Found 1150 images belonging to 7 classes.


In [7]:
train_image_data.image_shape
test_image_data.image_shape

(224, 224, 3)

In [8]:
# Model Architecting:
# CNN: A Convolution layer is combination of Convolve and Pooling:

model = tf.keras.Sequential()

# Conv2d(noofFeatureMaps, kernelShape, inputShape, activation) + padding = same -- add virtual boarder in every image
# NoOfFeatureMaps: Similar to no of units: Based on trial and error
# filter: (3,3), (4,4), (5,5) common :  in this content will be initialized randomly
# inputShape: similar to target_size

# ----------------- 1st convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape= train_image_data.image_shape, activation='relu', padding= 'same'))
# Adding Pooling:
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# ------------------- 2nd Convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding= 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# -------------------- Flatten:
model.add(tf.keras.layers.Flatten())

# FC Layer | ANN part, Layers creation:

model.add(tf.keras.layers.Dense(units= 256, activation='relu'))
model.add(tf.keras.layers.Dense(units= 512, activation='relu'))
model.add(tf.keras.layers.Dense(units= 128, activation='relu'))
model.add(tf.keras.layers.Dense(units= 256, activation= 'relu'))

# Output layer:
model.add(tf.keras.layers.Dense(units = 7, activation='softmax'))

# Model Summay:
model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,932,583 (98.92 MB)

 Trainable params: 25,932,583 (98.92 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# COmpilation:
# Balanced dat is there
# A balanced dataset, in the context of machine learning classification tasks, is a dataset in which every class or category is represented by an approximately equal number of samples. 
model.compile(optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
# Model training:

model.fit(train_image_data, 
          validation_data = test_image_data, 
          epochs = 100, 
          steps_per_epoch = (len(train_image_data.filenames)//train_image_data.batch_size ),
          validation_steps= (len(test_image_data.filenames)//test_image_data.batch_size),
          callbacks = MyCLRuleMonitor(0.9))
          

Epoch 1/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 585s 510ms/step - accuracy: 0.9117 - loss: 0.3216 - val_accuracy: 0.6597 - val_loss: 0.9680
Epoch 2/100
   1/1141 ━━━━━━━━━━━━━━━━━━━━ 8:09 430ms/step - accuracy: 0.3333 - loss: 1.7754

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1141/1141 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.3333 - loss: 1.7754 - val_accuracy: 0.6641 - val_loss: 0.9460
Epoch 3/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 514s 450ms/step - accuracy: 0.9623 - loss: 0.1530 - val_accuracy: 0.8721 - val_loss: 0.9520
Epoch 4/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 1.0000 - loss: 2.4914e-05 - val_accuracy: 0.8190 - val_loss: 2.1108
Epoch 5/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 476s 417ms/step - accuracy: 0.9921 - loss: 0.0333 - val_accuracy: 0.9843 - val_loss: 0.0520
Epoch 6/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9843 - val_loss: 0.0520
Epoch 7/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 523s 458ms/step - accuracy: 1.0000 - loss: 5.8128e-06 - val_accuracy: 0.9878 - val_loss: 0.0376
Epoch 8/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9878 - val_loss: 0.0376
Epoch 9/100
1141/1141 ━━━━━━━━━━━━━━━━━━━━ 40067s 35s/step - 

In [ ]:
# Input / Deployment / Testing:

image = tf.keras.preprocessing.image.load_img('fazli-mango.jpg', target_size = (224,224))
image_array = tf.keras.preprocessing.image.img_to_array(image)

image_np_array = np.expand_dims(image_array, axis = 0)
prediction_probabilities = model.predict(image_np_array)

train_dir = 'fruits-small/fruits-small/data/Training'
class_names = sorted(os.listdir(train_dir))

prediction_class_index= np.argmax(prediction_probabilities)
prediction_class_name = class_names[prediction_class_index]
print(prediction_class_index)
print(class_names)
print(prediction_class_name)

probability_prediction = model.predict(image_np_array)
# For binary classification with sigmoid output, we apply a threshold
# If probability_prediction[0][0] >= 0.5, it's class 1 (dogs), otherwise class 0 (cats)

if probability_prediction[0][0] >= 0.5:
    prediction_class_index = 1  # Corresponds to 'dogs'
else:
    prediction_class_index = 0  # Corresponds to 'cats'

print(f"Raw probability: {probability_prediction[0][0]:.4f}")

class_names_map = {v: k for k, v in train_image_data.class_indices.items()}

# Use the index to look up the actual class name
predicted_class_name = class_names_map[prediction_class_index]

# Print the result
print(f"The predicted pet name is: {predicted_class_name}")

NameError: name 'tf' is not defined

It is not able to predict a Mango and giving res as Banana, So WIll be playing with the batch size and let it run again, then we will try again